Inicialmente é preciso inserir as variáveis correspondentes às probabilidades já conhecidas:

In [2]:
# coding: utf-8
import pandas as pd
from IPython.display import display

# probabilidade de navegador corresponder ao perfil
P_N = pd.Series([0.4, 0.6], index=['N=1', 'N=2'])
print P_N

# probabilidade de navegador realizar compras ou não
P_C_N = pd.DataFrame({'V': [0.9, 0.3], 'F': [0.1, 0.7]}, index=['N=1', 'N=2'])
print P_C_N

P_DuTd_N = pd.DataFrame({  # probabilidades de acesso para cada tipo de navegador
    'Du=V,Td=V': [0.3, 0.3],
    'Du=V,Td=F': [0.1, 0.3],
    'Du=F,Td=V': [0.2, 0.3],
    'Du=F,Td=F': [0.4, 0.1],
}, index=['N=1', 'N=2'])
print P_DuTd_N

N=1    0.4
N=2    0.6
dtype: float64
       F    V
N=1  0.1  0.9
N=2  0.7  0.3
     Du=F,Td=F  Du=F,Td=V  Du=V,Td=F  Du=V,Td=V
N=1        0.4        0.2        0.1        0.3
N=2        0.1        0.3        0.3        0.3


A partir daí, podemos aplicar os classificadores bayesianos.

Para o classificador pela máxima probabilidade ``a posteriori``, é necessário calcular as probabilidades máximas para cada tipo de acesso:

In [3]:
P_N_DuTd = P_DuTd_N.apply(lambda col: col*P_N, axis=0)
print 'P(N|Du,Td):'
print P_N_DuTd

hmax = P_N_DuTd.idxmax(axis=0)
print 'h_max:'
print hmax

P_C_hmax = P_C_N.loc[hmax]
P_C_hmax.index = hmax.index
print 'P(C|h_max):'
print P_C_hmax

cmap = P_C_hmax.idxmax(axis=1)
print 'C_MAP:'
print cmap

P(N|Du,Td):
     Du=F,Td=F  Du=F,Td=V  Du=V,Td=F  Du=V,Td=V
N=1       0.16       0.08       0.04       0.12
N=2       0.06       0.18       0.18       0.18
h_max:
Du=F,Td=F    N=1
Du=F,Td=V    N=2
Du=V,Td=F    N=2
Du=V,Td=V    N=2
dtype: object
P(C|h_max):
             F    V
Du=F,Td=F  0.1  0.9
Du=F,Td=V  0.7  0.3
Du=V,Td=F  0.7  0.3
Du=V,Td=V  0.7  0.3
C_MAP:
Du=F,Td=F    V
Du=F,Td=V    F
Du=V,Td=F    F
Du=V,Td=V    F
dtype: object


Com isso temos os valores para a coluna C_{MAP}.

O classificador ótimo bayesiano usa uma fórmula similar:

In [5]:
P_C_V = P_N_DuTd.apply(lambda col: col*P_C_N['V'], axis=0).sum(axis=0)
P_C_F = P_N_DuTd.apply(lambda col: col*P_C_N['F'], axis=0).sum(axis=0)
P_Cj = pd.concat([P_C_V, P_C_F], axis=1, keys=['V', 'F'])

copt = P_Cj.idxmax(axis=1)
display(P_Cj, copt)

,V,F
"Du=F,Td=F",0.162,0.058
"Du=F,Td=V",0.126,0.134
"Du=V,Td=F",0.090,0.130
"Du=V,Td=V",0.162,0.138


Du=F,Td=F    V
Du=F,Td=V    F
Du=V,Td=F    F
Du=V,Td=V    V
dtype: object

Como pode-se ver na matriz de probabilidades resultante, o classificador ótimo bayesiano gera um empate para `Du = V` e `Td = V`.

Através do uso de ambos os classificadores, podemos obter os seguintes valores:


In [185]:
pd.concat([cmap, copt], axis=1, keys=['C_MAP', 'C_OPT'])

,C_MAP,C_OPT
"Du=F,Td=F",V,V
"Du=F,Td=V",F,F
"Du=V,Td=F",F,F
"Du=V,Td=V",F,V
